# Classifying cancer expression vectors

In this assignment you will train a neural network to identify the tissue type that produced an RNA expression vector. The dataset is comprised of RNA-seq data obtained from tumors. 

For a complete description of the data collection workflow see this page:
https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net

And for the corresponding publication:
https://doi.org/10.1038/nbt.3772

In [2]:
import sys
import numpy as np
from matplotlib import pyplot
import pandas as pd
import h5py
import os
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Loading and parsing training data
For this problem, expression data needs to be loaded and pruned. Initially, there are >50,000 genes in each expression vector, which can be reduced to a much smaller gene set for the sake of minimizing computation time. Here, the data is subsetted to only include genes from the KEGG gene set. You may want to consider reducing or expanding this dataset to get a better understanding of which genes are predictive, though this is not a requirement for the assignment.

For a list of gene sets, check out the MSigDB collections page: http://software.broadinstitute.org/gsea/msigdb/collections.jsp

This script was adapted from Rob Currie's ingestion script: https://github.com/rcurrie/tumornormal/blob/master/genesets.ipynb

In [3]:
np.random.seed(42)

print("Loading hdf files...")

tcga_h5_pathname = "data/tcga_target_gtex_train.h5"
msigdb_gmt_pathname = "data/c2.cp.kegg.v6.1.symbols.gmt"    # Needs a login to download from URL. Must be hosted locally

x = pd.read_hdf(tcga_h5_pathname, "expression")
x.head()

y = pd.read_hdf(tcga_h5_pathname, "labels")
y.head()

# Generate a set object from all of the TCGA gene symbols
tcga_gene_set = set(x.columns.values)

gene_sets = dict()

# Load gene sets from downloaded MSigDB gmt file (KEGG gene sets)
with open(msigdb_gmt_pathname) as file:
    for line in file:
        line = line.strip().split('\t')
        set_name = line[1]
        kegg_gene_subset = set(line[2:])

        # Find genes that are in both the KEGG database AND the TCGA dataset (AKA the intersection)
        genes = kegg_gene_subset & tcga_gene_set

        # Store the sets in their separate categories (just in case we want to subset by category later)
        gene_sets[set_name] = genes


print("Loaded %d gene sets" % len(gene_sets))

# Find the union of all genes in the gene sets
all_gene_set_genes = sorted(list(set.union(*[gene_set for gene_set in gene_sets.values()])))

print("Subsetting to %d genes" % len(all_gene_set_genes))

# Prune x so that it only contains the genes that are in both TCGA and KEGG
x_pruned = x.drop(labels=(set(x.columns)-set(all_gene_set_genes)), axis=1, errors="ignore")

assert x_pruned["TP53"]["TCGA-ZP-A9D4-01"] == x["TP53"]["TCGA-ZP-A9D4-01"]

m,n = x_pruned.shape                # m = number of training examples
print("x_pruned shape: ", m, n)

# Make sure the genes are the same and in the same order
assert len(all_gene_set_genes) == len(x_pruned.columns.values)
assert list(x_pruned.columns.values) == all_gene_set_genes

x_array = np.array(x_pruned.values, dtype=np.float32)

# extract primary site names from data table
y_names = list(set(y["primary_site"].values))
y_names = sorted(y_names)

y_array = np.zeros(shape=(m, len(y_names)), dtype=np.float32)

# create a key for converting names to indices
y_index_key = {name:i for i,name in enumerate(y_names)}

# generate one-hot vectors for all primary site y data
for m,primary_site_name in enumerate(y["primary_site"].values):
    index = y_index_key[primary_site_name]
    y_array[m,index] = 1

# split data into training and testing set with equal class representation
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(x_array, y_array):
    x_train, x_test = x_array[train_index], x_array[test_index]
    y_train, y_test = y_array[train_index], y_array[test_index]

print("Training set x dimensions: ", x_train.shape)
print("Training set y dimensions: ", y_train.shape)
print("Testing set x dimensions: ", x_test.shape)
print("Testing set y dimensions: ", y_test.shape)

# ensure that the output directory exists
if not os.path.exists("data/"):
    os.mkdir("data/")

# save the data in compressed numpy files
np.savez_compressed("data/x_train.npz", a=x_train)
np.savez_compressed("data/y_train.npz", a=y_train)
np.savez_compressed("data/x_test.npz", a=x_test)
np.savez_compressed("data/y_test.npz", a=y_test)

Loading hdf files...
Loaded 186 gene sets
Subsetting to 5172 genes
x_pruned shape:  19126 5172
Training set x dimensions:  (15300, 5172)
Training set y dimensions:  (15300, 46)
Testing set x dimensions:  (3826, 5172)
Testing set y dimensions:  (3826, 46)


## Define a pytorch Dataset object to contain the training and testing data
Pytorch handles data shuffling and batch loading, as long as the user provides a "Dataset" class. This class is just a wrapper for your data that casts the data into pytorch tensor format and returns slices of the data. In this case, our data has been stored in numpy format, which conveniently pytorch has a method for converting to their native format.

In [4]:
class PrimarySiteDataset(Dataset):
    def __init__(self, x_path, y_path, batch_size=None):
        x = np.load(x_path)['a']
        y = np.load(y_path)['a']

        x_dtype = torch.FloatTensor
        y_dtype = torch.FloatTensor     # for MSE Loss

        self.length = x.shape[0]

        self.x_data = torch.from_numpy(x).type(x_dtype)
        self.y_data = torch.from_numpy(y).type(y_dtype)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length